# **Imports:**

---



In [ ]:
# basic imports
import sys
import os
import csv
import random

# Pytorch libaries
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torchvision.models as models
from torch.utils.data import DataLoader, random_split, ConcatDataset, Subset
from torchvision.datasets import ImageFolder

# For loop
from timeit import default_timer as timer
from tqdm.auto import tqdm
from IPython.display import clear_output

# For plotting and visualizing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# For saving a path and loading
from pathlib import Path
import requests
from google.colab import drive
drive.mount('/content/drive')

print("Imported successfully")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Imported successfully


In [ ]:
print(f"Torch version: {torch.__version__}\n")
print(f"Torchvision version: {torchvision.__version__}\n")
print(f"CUDA version: {torch.version.cuda}\n")
print(f"cuDNN version: {torch.backends.cudnn.version()}\n")
print(f"Python version: {sys.version}")

Torch version: 2.2.1+cu121

Torchvision version: 0.17.1+cu121

CUDA version: 12.1

cuDNN version: 8906

Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


# **Training loop setup:**

---



In [ ]:
device = 'cuda' if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, download skipped")
else:
  print("helper_functions.py DNE, download initiated")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

from helper_functions import accuracy_fn, print_train_time

if 'accuracy_fn' and 'print_train_time' in dir():
    print("Both functions are imported.")
else:
    print("Accuracy function is not imported.")

helper_functions.py DNE, download initiated
Both functions are imported.


In [ ]:
def train_step(model: torch.nn.Module,
                   data_loader: torch.utils.data.DataLoader,
                   loss_fn: torch.nn.Module,
                   optimizer: torch.optim.Optimizer,
                   accuracy_fn,
                   device: torch.device = device):
  train_loss, train_acc = 0, 0
  # For training
  model.train()
  for batch, (image, label) in enumerate(data_loader):

    # Optimizer zero grad
    optimizer.zero_grad()

    # Forward pass
    image = image.to(device)
    label = label.to(device, dtype=torch.float)
    pred = model(image).squeeze()

    # Calculate the loss
    loss = loss_fn(pred, label)
    train_loss += loss
    train_acc += accuracy_fn(y_true = label,
                             y_pred = torch.round(torch.sigmoid(pred)))

    # Backprop
    loss.backward()

    # Optimizer step
    optimizer.step()

  # For calculating average trainloss over every batch in each epoch
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  print(f"Train loss: {train_loss: 5f} | Train acc: {train_acc:.2f}")

In [ ]:
def test_step(model: torch.nn.Module,
                  data_loader: torch.utils.data.DataLoader,
                  loss_fn: torch.nn.Module,
                  optimizer: torch.optim.Optimizer,
                  accuracy_fn,
                  device: torch.device = device):

  test_loss, test_acc = 0, 0
  model.eval()

  with torch.inference_mode():
    for batch, (image, label) in enumerate(data_loader):
      # Forward pass
      image = image.to(device)
      label = label.to(device, dtype=torch.float)
      pred = model(image).squeeze()

      # Loss calculation
      loss = loss_fn(pred, label)
      test_loss += loss

      # Accuracy calculation
      test_acc += accuracy_fn(y_true = label,
                              y_pred = torch.round(torch.sigmoid(pred)))

    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
    print(f"Test loss: {test_loss: 5f} | Test acc: {test_acc:.2f}")

# **Model setup:**

---



In [ ]:
model = models.resnet34(weights='IMAGENET1K_V1')
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = model.fc.in_features
num_classes = 1
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 152MB/s]


# **Training loop:**

---



In [ ]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.001)
loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
def load_dataset(dataset_path, transform=None):
    return ImageFolder(dataset_path, transform=transform)

# Define your transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Grayscale(num_output_channels=1)
])

combined_dataset = load_dataset('/content/drive/MyDrive/dogorcat', transform=transform)

print(f"Class types: {combined_dataset.classes}")

subset_indices = random.sample(range(len(combined_dataset)), 5500)
subset_dataset = Subset(combined_dataset, subset_indices)

# Split the subset into train and test sets
train_size = int(0.8 * len(subset_dataset))  # 80% for training
test_size = len(subset_dataset) - train_size  # 20% for testing
train_dataset, test_dataset = random_split(subset_dataset, [train_size, test_size])

# Create DataLoaders for train and test sets
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

Class types: ['Cat', 'Dog']


In [ ]:
epochCount = []
trainLosses = []
testLosses = []
accuracyCount = []

epochs = 5
timeStart = timer()

for epoch in tqdm(range(epochs)):
  train_step(model = model,
             data_loader = train_dataloader,
             loss_fn = loss_fn,
             optimizer = optimizer,
             accuracy_fn = accuracy_fn,
             device = device)

  test_step(model = model,
            data_loader = test_dataloader,
            loss_fn = loss_fn,
            optimizer = optimizer,
            accuracy_fn = accuracy_fn,
            device = device)

timeEnd = timer()
totalTrainTime = print_train_time(start = timeStart,
                                  end = timeEnd,
                                  device = str(next(model.parameters()).device))

# **Model saving:**

---

In [ ]:
# Set the path in Google Drive
drive_path = Path('/content/drive/My Drive/models')
drive_path.mkdir(parents=True, exist_ok=True)

# Create a model save path
modelName = 'catdog.pth'
modelSavePath = drive_path / modelName

print(f"SAVING MODEL TO: {modelSavePath}")

# Example model save (replace `model.state_dict()` with your actual model's state dict)
torch.save(obj=model.state_dict(), f=modelSavePath)


In [ ]:
# Create a model directory
modelPath = Path("models")
modelPath.mkdir(parents= True, exist_ok= True)

# Create a model savepath
modelName = input("Input model path name: ")

modelSavePath = modelPath / modelName

print(f"SAVING MODEL TO: {modelSavePath}")

torch.save(obj= model.state_dict(), f= modelSavePath)